## Import libraries

In [29]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Data Preprocessing

In [55]:
# define file name
filename = r'Use this dataset\uswtdb_v5_0_20220427_rename.csv'

# read csv file
wind_turbine_df = pd.read_csv(filename)

# check number of rows and columns
wind_turbine_df.shape

C:\Users\quakl\AppData\Local\Temp\ipykernel_21624\587385051.py:5: DtypeWarning: Columns (5,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90) have mixed types. Specify dtype option on import or set low_memory=False.
  wind_turbine_df = pd.read_csv(filename)


(80103, 124)

In [56]:
# get the columns of the dataframe
wind_turbine_df.columns

Index(['Plant Id', 'Combined Heat And\nPower Plant', 'Nuclear Unit Id',
       'Plant Name', 'Operator Name', 'Operator Id', 'Plant State',
       'Census Region', 'NERC Region', 'Reserved',
       ...
       'rotor_swept area (m2)', 'max_height (m)', 'retrofit (y/n)',
       'retrofit_year', 'attribute_confidence_level (1/2/3)',
       'location_confidence_level (1/2/3)', 'location_verification_date',
       'image_source', 'xlong', 'ylat'],
      dtype='object', length=124)

In [57]:
# filter unecessary columns
wind_turbine_df = wind_turbine_df[[#'faa_digital_obstacle_id', 'faa_obstruction_evaluation_id', 'usgs_id', 
                                   'eia_id', 'state', 'county',
                                   # 'state_county_id',
                                   'project_name', 'operational_year', 'project_turbine_num',
                                   'cumulative_capacity (MW)',
                                   # 'manufacturer', 'model_name', 
                                   'turbine_capacity (KW)',
                                   # 'hub_height (m)', 
                                   # 'rotor_diameter (m)', 'rotor_swept area (m2)', 'max_height (m)', 'retrofit (y/n)',
                                   # 'retrofit_year', 'attribute_confidence_level (1/2/3)',
                                   # 'location_confidence_level (1/2/3)', 'location_verification_date', 'image_source', 
                                   'xlong', 'ylat']]

# check for duplicates
wind_turbine_df.duplicated().sum()

15654

In [58]:
# remove duplicates
wind_turbine_df = wind_turbine_df.drop_duplicates()

# check number of rows and columns
wind_turbine_df.shape

(64449, 10)

In [59]:
# check null values
wind_turbine_df.isnull().sum()

eia_id                        1
state                         1
county                        1
project_name                  1
operational_year              3
project_turbine_num           1
cumulative_capacity (MW)      3
turbine_capacity (KW)       756
xlong                         1
ylat                          1
dtype: int64

In [83]:
# remove eia_id null values
wind_turbine_df = wind_turbine_df[wind_turbine_df['eia_id'].notna()]

# check null values
wind_turbine_df.isnull().sum()[lambda x: x > 0]

cumulative_capacity (MW)      2
turbine_capacity (KW)       755
dtype: int64

In [68]:
# check data types
wind_turbine_df.dtypes

eia_id                      float64
state                        object
county                       object
project_name                 object
operational_year            float64
project_turbine_num         float64
cumulative_capacity (MW)    float64
turbine_capacity (KW)       float64
xlong                       float64
ylat                        float64
dtype: object

In [70]:
# check eia_id using is_integer function
wind_turbine_df['eia_id'].apply(lambda x: pd.api.types.is_integer(x)).sum()

0

In [72]:
# change eia_id data type to integer
wind_turbine_df['eia_id'] = wind_turbine_df['eia_id'].astype('int64')

# check operational_year using is_integer function
wind_turbine_df['operational_year'].apply(lambda x: pd.api.types.is_integer(x)).sum()

0

In [74]:
# show operational_year null values
wind_turbine_df[wind_turbine_df['operational_year'].isnull()]

,eia_id,state,county,project_name,operational_year,project_turbine_num,cumulative_capacity (MW),turbine_capacity (KW),xlong,ylat
6733,50532,CA,Kern County,Historic Victory Garden (Wind Stream),NaN,215.0,17.0,NaN,-118.375799,35.074235
6777,50532,CA,Kern County,Historic Victory Garden (Wind Stream),NaN,215.0,17.0,NaN,-118.375610,35.074281


In [75]:
# replace project_name = "Historic Victory Garden (Wind Stream)" with operational_year = 1982
# http://www.windstreamproperties.com/victory-garden.html - proj details
# https://eerscmap.usgs.gov/uswtdb/viewer/#13.39/35.06889/-118.38267 - proj operational date
wind_turbine_df.loc[wind_turbine_df['project_name'] == 'Historic Victory Garden (Wind Stream)', 'operational_year'] = 1982

# convert operational_year to integer
wind_turbine_df['operational_year'] = wind_turbine_df['operational_year'].astype('int64')

# check project_turbine_num using is_integer function
wind_turbine_df['project_turbine_num'].apply(lambda x: pd.api.types.is_integer(x)).sum()

0

In [77]:
# convert project_turbine_num to integer
wind_turbine_df['project_turbine_num'] = wind_turbine_df['project_turbine_num'].astype('int64')

# check cumulative_capacity (MW) using is_integer function
wind_turbine_df['cumulative_capacity (MW)'].apply(lambda x: pd.api.types.is_integer(x)).sum()

0

In [80]:
# show cumulative_capacity (MW) null values
wind_turbine_df[wind_turbine_df['cumulative_capacity (MW)'].isnull()]

,eia_id,state,county,project_name,operational_year,project_turbine_num,cumulative_capacity (MW),turbine_capacity (KW),xlong,ylat
71342,62442,MN,Freeborn County,"Lake Region Community Hybrid, LLC",2018,1,NaN,NaN,-93.307907,43.580891
73015,62642,TX,Lubbock County,unknown Lubbock County,2015,6,NaN,NaN,-102.056847,33.591496


In [81]:
# show eia_id = 62642
wind_turbine_df[wind_turbine_df['eia_id'] == 62642]

,eia_id,state,county,project_name,operational_year,project_turbine_num,cumulative_capacity (MW),turbine_capacity (KW),xlong,ylat
73015,62642,TX,Lubbock County,unknown Lubbock County,2015,6,NaN,NaN,-102.056847,33.591496


In [79]:
# convert cumulative_capacity (MW) to integer
wind_turbine_df['cumulative_capacity (MW)'] = wind_turbine_df['cumulative_capacity (MW)'].astype('int64')

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [62]:
# filter for both cumulative_capacity (MW) and turbine_capacity (KW)  = null
wind_turbine_df[wind_turbine_df['cumulative_capacity (MW)'].isnull() & wind_turbine_df['turbine_capacity (KW)'].isnull()].shape[0]


2

In [61]:
# show only turbine_capacity (KW) = null and cumulative_capacity (MW) = null
wind_turbine_df[(wind_turbine_df['turbine_capacity (KW)'].isnull()) & (wind_turbine_df['cumulative_capacity (MW)'].isnull())]

,eia_id,state,county,project_name,operational_year,project_turbine_num,cumulative_capacity (MW),turbine_capacity (KW),xlong,ylat
71342,62442.0,MN,Freeborn County,"Lake Region Community Hybrid, LLC",2018.0,1.0,NaN,NaN,-93.307907,43.580891
73015,62642.0,TX,Lubbock County,unknown Lubbock County,2015.0,6.0,NaN,NaN,-102.056847,33.591496


In [ ]:
# filter eia_id = 62642.0

In [67]:
# show turbine_capacity (KW) = null and cumulative_capacity (MW) != null
wind_turbine_df[(wind_turbine_df['turbine_capacity (KW)'].isnull()) & (wind_turbine_df['cumulative_capacity (MW)'].notnull())].shape

(753, 10)

In [64]:
# show eia_id = 10597.0 and project_turbine_num = 236.0	
wind_turbine_df[(wind_turbine_df['eia_id'] == 10597.0) & (wind_turbine_df['project_turbine_num'] == 236.0)]

,eia_id,state,county,project_name,operational_year,project_turbine_num,cumulative_capacity (MW),turbine_capacity (KW),xlong,ylat
5482,10597.0,CA,Kern County,Ridgetop,1985.0,236.0,29.0,NaN,-118.303085,35.081696
5483,10597.0,CA,Kern County,Ridgetop,1985.0,236.0,29.0,NaN,-118.295344,35.077554
5484,10597.0,CA,Kern County,Ridgetop,1985.0,236.0,29.0,NaN,-118.328491,35.076195
5485,10597.0,CA,Kern County,Ridgetop,1985.0,236.0,29.0,NaN,-118.304154,35.077404
5486,10597.0,CA,Kern County,Ridgetop,1985.0,236.0,29.0,NaN,-118.331490,35.074795
...,...,...,...,...,...,...,...,...,...,...
5692,10597.0,CA,Kern County,Ridgetop,1985.0,236.0,29.0,NaN,-118.307396,35.073655
5693,10597.0,CA,Kern County,Ridgetop,1985.0,236.0,29.0,NaN,-118.306786,35.074097
5694,10597.0,CA,Kern County,Ridgetop,1985.0,236.0,29.0,NaN,-118.326286,35.073696
5695,10597.0,CA,Kern County,Ridgetop,1985.0,236.0,29.0,NaN,-118.324989,35.073296


In [66]:
# check if the project_turbine_num equals to the count of each eia_id
wind_turbine_df.groupby('eia_id')['project_turbine_num'].nunique()[lambda x: x > 0].sort_values(ascending=False)


eia_id
57833.0    6
57294.0    5
56252.0    4
6304.0     4
57291.0    4
          ..
55574.0    1
55566.0    1
57296.0    1
57299.0    1
57744.0    1
Name: project_turbine_num, Length: 1172, dtype: int64

In [149]:
# show the unique values for turbine number for eia_id = 57833.0
wind_turbine_df[wind_turbine_df['eia_id'] == 57833.0]\
    [['eia_id','project_name', 'project_turbine_num', 'cumulative_capacity (MW)', 'turbine_capacity (KW)']]\
    .drop_duplicates()

,eia_id,project_name,project_turbine_num,cumulative_capacity (MW),turbine_capacity (KW)
394,57833.0,Alta II,50,150.0,3000.0
487,57833.0,Alta IV,34,102.0,3000.0
556,57833.0,Alta V,56,168.0,3000.0
602,57833.0,Alta VI (partial),44,132.0,3000.0
645,57833.0,Alta VIII (partial),43,129.0,3000.0
42034,57833.0,Mustang Hills,6,18.0,3000.0


In [157]:
# check if the cumulative_capacity (MW) equals to the count of each eia_id
wind_turbine_df.groupby('eia_id')['cumulative_capacity (MW)'].nunique()[lambda x: x > 0].sort_values(ascending=False)

eia_id
57833.0    6
57294.0    5
6304.0     4
57291.0    4
56357.0    4
          ..
57239.0    1
57240.0    1
57244.0    1
57251.0    1
65525.0    1
Name: cumulative_capacity (MW), Length: 1254, dtype: int64

In [172]:
# concatenate the eia_id and project_name
wind_turbine_df['eia_id_project_name'] = wind_turbine_df['eia_id'].astype(str) + '_' + wind_turbine_df['project_name'] #+ '_' + wind_turbine_df['location_verification_date'].astype(str)

# show the unique values for eia_id_project_name
wind_turbine_df.groupby('eia_id_project_name')['cumulative_capacity (MW)'].nunique()[lambda x: x > 1].sort_values(ascending=False)

eia_id_project_name
57526.0_South Hurlburt                  4
56357.0_Wilton Wind Farm                4
56252.0_Century Expansion               3
56925.0_Crystal Lake II                 3
56843.0_Seven Mile Hill I & II          3
                                       ..
57195.0_Lower Snake River Phase I       2
50281.0_San Gorgonio Farms Wind Farm    2
57296.0_Carleton College                2
57332.0_Crossroads                      2
nan_unknown Riverside County            2
Name: cumulative_capacity (MW), Length: 66, dtype: int64

In [173]:
# show the unique values for eia_id_project_name equals to '56357.0_Wilton Wind Farm'
wind_turbine_df[wind_turbine_df['eia_id_project_name'] == '57526.0_South Hurlburt']\
[['eia_id_project_name','eia_id','project_name', 'project_turbine_num', 'cumulative_capacity (MW)', 'turbine_capacity (KW)', 'operational_year']]\
.drop_duplicates()

,eia_id_project_name,eia_id,project_name,project_turbine_num,cumulative_capacity (MW),turbine_capacity (KW),operational_year
58696,57526.0_South Hurlburt,57526.0,South Hurlburt,82,224.75,2750.0,2011.0
58697,57526.0_South Hurlburt,57526.0,South Hurlburt,23,63.25,2750.0,2012.0
58699,57526.0_South Hurlburt,57526.0,South Hurlburt,82,224.75,2500.0,2011.0
58757,57526.0_South Hurlburt,57526.0,South Hurlburt,116,290.00,2500.0,2012.0
58803,57526.0_South Hurlburt,57526.0,South Hurlburt,82,205.00,2500.0,2011.0


In [167]:
wind_turbine_df.columns

Index(['case_id', 'faa_digital_obstacle_id', 'faa_obstruction_evaluation_id',
       'usgs_id', 'eia_id', 'state', 'county', 'state_county_id',
       'project_name', 'operational_year', 'project_turbine_num',
       'cumulative_capacity (MW)', 'manufacturer', 'model_name',
       'turbine_capacity (KW)', 'hub_height (m)', 'rotor_diameter (m)',
       'rotor_swept area (m2)', 'max_height (m)', 'retrofit (y/n)',
       'retrofit_year', 'attribute_confidence_level (1/2/3)',
       'location_confidence_level (1/2/3)', 'location_verification_date',
       'image_source', 'xlong', 'ylat', 'eia_id_project_name'],
      dtype='object')

In [197]:
pd.set_option('display.max_columns', None)
wind_turbine_df[wind_turbine_df['eia_id_project_name'] == '57526.0_South Hurlburt']\
[['eia_id',
 'project_name',
 'project_turbine_num',
 'cumulative_capacity (MW)',
 'retrofit (y/n)',
 'retrofit_year',
 'eia_id_project_name']].drop_duplicates().sort_values(by=['cumulative_capacity (MW)'], ascending=False)

,eia_id,project_name,project_turbine_num,cumulative_capacity (MW),retrofit (y/n),retrofit_year,eia_id_project_name
58757,57526.0,South Hurlburt,116,290.00,0,NaN,57526.0_South Hurlburt
58696,57526.0,South Hurlburt,82,224.75,1,2021.0,57526.0_South Hurlburt
58803,57526.0,South Hurlburt,82,205.00,1,2021.0,57526.0_South Hurlburt
58697,57526.0,South Hurlburt,23,63.25,1,2021.0,57526.0_South Hurlburt


In [176]:
# list of all the columns
list(wind_turbine_df.columns)

['case_id',
 'faa_digital_obstacle_id',
 'faa_obstruction_evaluation_id',
 'usgs_id',
 'eia_id',
 'state',
 'county',
 'state_county_id',
 'project_name',
 'operational_year',
 'project_turbine_num',
 'cumulative_capacity (MW)',
 'manufacturer',
 'model_name',
 'turbine_capacity (KW)',
 'hub_height (m)',
 'rotor_diameter (m)',
 'rotor_swept area (m2)',
 'max_height (m)',
 'retrofit (y/n)',
 'retrofit_year',
 'attribute_confidence_level (1/2/3)',
 'location_confidence_level (1/2/3)',
 'location_verification_date',
 'image_source',
 'xlong',
 'ylat',
 'eia_id_project_name']